In [197]:
###libraries we need for data analysis
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
import sys

In [3]:
!pip install git+https://github.com/simfin/simfin.git

  Cloning https://github.com/simfin/simfin.git to /private/var/folders/zb/w1cyxmx94h3g86hskcv8p08h0000gn/T/pip-req-build-cmutl15b
  Running setup.py bdist_wheel for simfin ... done
  Stored in directory: /private/var/folders/zb/w1cyxmx94h3g86hskcv8p08h0000gn/T/pip-ephem-wheel-cache-794_ho0e/wheels/dc/57/64/e9165288c478f38b172239c9f9712db2d0fcbc980e7357a3d8
Successfully built simfin


In [73]:
import simfin as sf
from simfin.names import *
sf.set_api_key('free')
sf.set_data_dir('~/simfin_data/') #set working dictionary
stock_df =  sf.load_shareprices(variant='daily', market='us')  #us daily share price 

Dataset "us-shareprices-daily" on disk (24 days old).
- Loading from disk ... Done!


In [109]:
stock_df=pd.read_csv('/Users/xinrucheng/Desktop/capstone/us-shareprices-daily.csv',sep=';')

In [110]:
stock_df.shape # 5226627 rows and 8 columns 
stock_df.head()

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume
0,GOOG,18,2007-01-03,115.9490,114.7323,118.6014,116.3447,232.92,NaN,15470700
1,GOOG,18,2007-01-04,116.6955,116.5337,120.4153,120.2436,240.73,NaN,15834200
2,GOOG,18,2007-01-05,120.0545,118.9622,121.2986,121.2214,242.69,NaN,13795600
3,GOOG,18,2007-01-08,121.3459,119.9799,121.8883,120.3232,240.89,NaN,9544400
4,GOOG,18,2007-01-09,120.7885,119.7310,121.4852,120.8009,241.84,NaN,10803000


In [111]:
stock_df['SimFinId'].nunique() # 2026 company 
stock_df.columns[stock_df.isnull().any()].tolist() # only dividend has null values 

['Open', 'Low', 'High', 'Close', 'Adj. Close', 'Dividend']

In [112]:
stock_df['Dividend'].fillna(0, inplace = True)

In [113]:
stock_df=stock_df.reset_index(level=['Date','Ticker'])

KeyError: 'Level Date must be same as name (None)'

In [114]:
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

In [115]:
stock_df['Date'].describe()

count                 5099376
unique                   3077
top       2017-07-10 00:00:00
freq                     1934
first     2007-01-03 00:00:00
last      2019-03-25 00:00:00
Name: Date, dtype: object

In [276]:
stock2=stock_df[['Ticker','Date','Open','Close','Dividend']]
stock2['year']=stock2['Date'].dt.year

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# remove rows with company that only has one stock price in a certain year

In [277]:
#DOMO 2008
#DOMO 2009
#FCS 2015
#FRX 2015
#MFON 2010

In [316]:
sum(stock2[(stock2['Ticker']=='A') & (stock2['year']==2012)]['Dividend'])

0.4

In [278]:
indexNames1 = stock2[ (stock2.Ticker == 'DOMO') & (stock2.year == 2008)].index
stock2.drop(indexNames1 , inplace=True)
indexNames2 = stock2[ (stock2.Ticker == 'DOMO') & (stock2.year == 2009)].index
stock2.drop(indexNames2 , inplace=True)
indexNames3 = stock2[ (stock2.Ticker == 'FCS') & (stock2.year == 2015)].index
stock2.drop(indexNames3 , inplace=True)
indexNames4 = stock2[ (stock2.Ticker == 'FRX') & (stock2.year == 2015)].index
stock2.drop(indexNames4 , inplace=True)
indexNames5 = stock2[ (stock2.Ticker == 'MFON') & (stock2.year == 2010)].index
stock2.drop(indexNames5 , inplace=True)

In [263]:
#stock3=stock2[(stock2.Ticker != 'DOMO') & (stock2.year != 2008)]
#stock3=stock3[(stock3.Ticker != 'DOMO') & (stock3.year != 2009)]
#stock3=stock3[(stock3.Ticker != 'FCS') & (stock3.year != 2015)]
#stock3=stock3[(stock3.Ticker != 'FRX') & (stock3.year != 2015)]
#stock3=stock3[(stock3.Ticker != 'MFON') & (stock3.year != 2010)]

In [279]:
#first_last.loc[first_last['Ticker']=='DOMO',]
stock2.loc[stock2['Ticker']=='DOMO',]

,Ticker,Date,Open,Close,Dividend,year
4493624,DOMO,2012-10-03,18.0000,18.00,0.0,2012
4493625,DOMO,2012-10-04,18.0000,18.00,0.0,2012
4493626,DOMO,2012-10-05,18.0000,18.00,0.0,2012
4493627,DOMO,2012-10-08,18.0000,18.00,0.0,2012
4493628,DOMO,2012-10-09,18.0000,18.00,0.0,2012
4493629,DOMO,2012-10-10,18.0000,18.00,0.0,2012
4493630,DOMO,2012-10-11,18.0000,18.00,0.0,2012
4493631,DOMO,2012-10-12,18.0000,18.00,0.0,2012
4493632,DOMO,2012-10-15,18.0000,18.00,0.0,2012
4493633,DOMO,2012-10-16,18.0000,18.00,0.0,2012


In [280]:
#stock3.head()
#stock3.describe()
#stock2.loc[stock2['Ticker']=='DOMO',]
#first_last.loc[first_last['Ticker']=='DOMO',]

In [281]:
##get the beginning and last entries of each Ticker and Year
df = stock2.groupby(['Ticker','year'])
first_last=(pd.concat([df.head(1), df.tail(1)])
   .sort_values(['Ticker','Date'])
            .drop_duplicates()
   .reset_index(drop=True))
first_last.head()

,Ticker,Date,Open,Close,Dividend,year
0,A,2007-01-03,34.99,34.30,0.0,2007
1,A,2007-12-31,37.30,36.74,0.0,2007
2,A,2008-01-02,36.67,36.30,0.0,2008
3,A,2008-12-31,14.92,15.63,0.0,2008
4,A,2009-01-02,15.60,16.24,0.0,2009


In [320]:
# get dividend data frame
dividend=pd.DataFrame(stock2.groupby(['Ticker','year']).sum()['Dividend']).reset_index()
dividend

,Ticker,year,Dividend
0,A,2007,0.000
1,A,2008,0.000
2,A,2009,0.000
3,A,2010,0.000
4,A,2011,0.000
5,A,2012,0.400
6,A,2013,0.492
7,A,2014,16.146
8,A,2015,0.515
9,A,2016,0.477


In [364]:
stock_return0=first_last[['Ticker','year']].drop_duplicates().reset_index(drop=True)
stock_return0['Return'] = np.nan
stock_return=dividend.merge(stock_return0,on=['Ticker','year'])
stock_return

,Ticker,year,Dividend,Return
0,A,2007,0.000,NaN
1,A,2008,0.000,NaN
2,A,2009,0.000,NaN
3,A,2010,0.000,NaN
4,A,2011,0.000,NaN
5,A,2012,0.400,NaN
6,A,2013,0.492,NaN
7,A,2014,16.146,NaN
8,A,2015,0.515,NaN
9,A,2016,0.477,NaN


In [360]:
thisTicker = stock_return['Ticker'][0]
thisYear = stock_return['year'][0]
open = float(first_last.loc[(first_last['Ticker']==thisTicker) & (first_last['year']==thisYear), "Open"].iloc[0])
close = first_last.loc[(first_last['Ticker']==thisTicker) & (first_last['year']==thisYear), "Close"].iloc[1]
div = stock_return['Dividend'][0]
stock_return['Return'][0] = (close - open + div) / open
stock_return['Return']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0        0.050014
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
6             NaN
7             NaN
8             NaN
9             NaN
10            NaN
11            NaN
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17            NaN
18            NaN
19            NaN
20            NaN
21            NaN
22            NaN
23            NaN
24            NaN
25            NaN
26            NaN
27            NaN
28            NaN
29            NaN
           ...   
22166         NaN
22167         NaN
22168         NaN
22169         NaN
22170         NaN
22171         NaN
22172         NaN
22173         NaN
22174         NaN
22175         NaN
22176         NaN
22177         NaN
22178         NaN
22179         NaN
22180         NaN
22181         NaN
22182         NaN
22183         NaN
22184         NaN
22185         NaN
22186         NaN
22187         NaN
22188         NaN
22189         NaN
22190     

In [361]:
for i in range(0,len(stock_return)-1):
    thisTicker = stock_return['Ticker'][i]
    thisYear = stock_return['year'][i]
    open = float(first_last.loc[(first_last['Ticker']==thisTicker) & (first_last['year']==thisYear), "Open"].iloc[0])
    close = first_last.loc[(first_last['Ticker']==thisTicker) & (first_last['year']==thisYear), "Close"].iloc[1]
    div = stock_return['Dividend'][i]
    stock_return['Return'][i] = (close - open + div) / open
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [362]:
stock_return=stock_return[['Ticker','year','Return']]
stock_return

,Ticker,year,Return
0,A,2007,0.050014
1,A,2008,-0.573766
2,A,2009,0.991667
3,A,2010,0.319847
4,A,2011,-0.159528
5,A,2012,0.153460
6,A,2013,0.367844
7,A,2014,-0.000245
8,A,2015,0.027805
9,A,2016,0.121213


In [363]:
stock_return.to_csv('/Users/xinrucheng/Desktop/capstone/stock_return.csv', index = False)

# 2007 data

In [101]:
close2007=stock_df[stock_df['Date']=='2017-12-31'][['Ticker','Close']]
open2007=stock_df[stock_df['Date']=='2017-01-03'][['Ticker','Open']]
div2007=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2007].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2007=open2007.merge(close2007,on='Ticker')
stock2007=oc2007.merge(div2007,on='Ticker')
stock2007['return']=((stock2007['Close']-stock2007['Open'])+stock2007['Dividend'])/stock2007['Open']
close2007
#dividend = stock_df[stock_df['Date'].dt.year==2007]
#dividend2007=dividend.groupby(dividend['Ticker'])['Dividend'].sum()

#df_2008=all_df[(all_df['Date'].dt.year == 2008) & (all_df['Date'].dt.month == 10)] 
#go=df_2008['Open'].groupby(df_2008['Date'])

,Ticker,Close


# 2008 data

In [85]:
close2008=stock_df[stock_df['Date']=='2008-12-31'][['Ticker','Close']]
open2008=stock_df[stock_df['Date']=='2008-01-02'][['Ticker','Open']]
div2008=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2008].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2008=open2008.merge(close2008,on='Ticker')
stock2008=oc2008.merge(div2008,on='Ticker')
stock2008['2008_return']=((stock2008['Close']-stock2008['Open'])+stock2008['Dividend'])/stock2008['Open']
st08=pd.DataFrame(stock2008[['Ticker','2008_return']])

# 2009 data

In [86]:
close2009=stock_df[stock_df['Date']=='2009-12-31'][['Ticker','Close']]
open2009=stock_df[stock_df['Date']=='2009-01-02'][['Ticker','Open']]
div2009=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2009].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2009=open2009.merge(close2009,on='Ticker')
stock2009=oc2009.merge(div2009,on='Ticker')
stock2009['2009_return']=((stock2009['Close']-stock2009['Open'])+stock2009['Dividend'])/stock2009['Open']
st09=pd.DataFrame(stock2009[['Ticker','2009_return']])

# 2010 data

In [87]:
close2010=stock_df[stock_df['Date']=='2010-12-31'][['Ticker','Close']]
open2010=stock_df[stock_df['Date']=='2010-01-04'][['Ticker','Open']]
div2010=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2010].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2010=open2010.merge(close2010,on='Ticker')
stock2010=oc2010.merge(div2010,on='Ticker')
stock2010['2010_return']=((stock2010['Close']-stock2010['Open'])+stock2010['Dividend'])/stock2010['Open']
st10=pd.DataFrame(stock2010[['Ticker','2010_return']])

# 2011 data

In [88]:
close2011=stock_df[stock_df['Date']=='2011-12-30'][['Ticker','Close']]
open2011=stock_df[stock_df['Date']=='2011-01-03'][['Ticker','Open']]
div2011=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2011].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2011=open2011.merge(close2011,on='Ticker')
stock2011=oc2011.merge(div2011,on='Ticker')
stock2011['2011_return']=((stock2011['Close']-stock2011['Open'])+stock2011['Dividend'])/stock2011['Open']
st11=pd.DataFrame(stock2011[['Ticker','2011_return']])

# 2012 data

In [89]:
close2012=stock_df[stock_df['Date']=='2012-12-31'][['Ticker','Close']]
open2012=stock_df[stock_df['Date']=='2012-01-03'][['Ticker','Open']]
div2012=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2012].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2012=open2012.merge(close2012,on='Ticker')
stock2012=oc2012.merge(div2012,on='Ticker')
stock2012['2012_return']=((stock2012['Close']-stock2012['Open'])+stock2012['Dividend'])/stock2012['Open']
st12=pd.DataFrame(stock2012[['Ticker','2012_return']])

# 2013 data

In [90]:
close2013=stock_df[stock_df['Date']=='2013-12-31'][['Ticker','Close']]
open2013=stock_df[stock_df['Date']=='2013-01-02'][['Ticker','Open']]
div2013=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2013].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2013=open2013.merge(close2013,on='Ticker')
stock2013=oc2013.merge(div2013,on='Ticker')
stock2013['2013_return']=((stock2013['Close']-stock2013['Open'])+stock2013['Dividend'])/stock2013['Open']
st13=pd.DataFrame(stock2013[['Ticker','2013_return']])

# 2014 data

In [91]:
close2014=stock_df[stock_df['Date']=='2014-12-31'][['Ticker','Close']]
open2014=stock_df[stock_df['Date']=='2014-01-02'][['Ticker','Open']]
div2014=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2014].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2014=open2014.merge(close2014,on='Ticker')
stock2014=oc2014.merge(div2014,on='Ticker')
stock2014['2014_return']=((stock2014['Close']-stock2014['Open'])+stock2014['Dividend'])/stock2014['Open']
st14=pd.DataFrame(stock2014[['Ticker','2014_return']])

# 2015 data

In [92]:
close2015=stock_df[stock_df['Date']=='2015-12-31'][['Ticker','Close']]
open2015=stock_df[stock_df['Date']=='2015-01-02'][['Ticker','Open']]
div2015=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2015].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2015=open2015.merge(close2015,on='Ticker')
stock2015=oc2015.merge(div2015,on='Ticker')
stock2015['2015_return']=((stock2015['Close']-stock2015['Open'])+stock2015['Dividend'])/stock2015['Open']
st15=pd.DataFrame(stock2015[['Ticker','2015_return']])

# 2016 data

In [93]:
close2016=stock_df[stock_df['Date']=='2016-12-30'][['Ticker','Close']]
open2016=stock_df[stock_df['Date']=='2016-01-04'][['Ticker','Open']]
div2016=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2016].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2016=open2016.merge(close2016,on='Ticker')
stock2016=oc2016.merge(div2016,on='Ticker')
stock2016['2016_return']=((stock2016['Close']-stock2016['Open'])+stock2016['Dividend'])/stock2016['Open']
st16=pd.DataFrame(stock2016[['Ticker','2016_return']])

# 2017 data

In [94]:
close2017=stock_df[stock_df['Date']=='2017-12-29'][['Ticker','Close']]
open2017=stock_df[stock_df['Date']=='2017-01-03'][['Ticker','Open']]
div2017=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2017].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2017=open2017.merge(close2017,on='Ticker')
stock2017=oc2017.merge(div2017,on='Ticker')
stock2017['2017_return']=((stock2017['Close']-stock2017['Open'])+stock2017['Dividend'])/stock2017['Open']
st17=pd.DataFrame(stock2017[['Ticker','2017_return']])

# 2018 data

In [95]:
close2018=stock_df[stock_df['Date']=='2018-12-31'][['Ticker','Close']]
open2018=stock_df[stock_df['Date']=='2018-01-02'][['Ticker','Open']]
div2018=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2018].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2018=open2018.merge(close2018,on='Ticker')
stock2018=oc2018.merge(div2018,on='Ticker')
stock2018['2018_return']=((stock2018['Close']-stock2018['Open'])+stock2018['Dividend'])/stock2018['Open']
st18=pd.DataFrame(stock2018[['Ticker','2018_return']])

# 2019 data

In [96]:
close2019=stock_df[stock_df['Date']=='2019-03-01'][['Ticker','Close']] #last day is 03/01 for 2019
open2019=stock_df[stock_df['Date']=='2019-01-02'][['Ticker','Open']] 
div2019=pd.DataFrame(stock_df[stock_df['Date'].dt.year==2019].groupby(stock_df['Ticker']).sum()['Dividend'])
oc2019=open2019.merge(close2019,on='Ticker')
stock2019=oc2019.merge(div2019,on='Ticker')
stock2019['2019_return']=((stock2019['Close']-stock2019['Open'])+stock2019['Dividend'])/stock2019['Open']
st19=pd.DataFrame(stock2019[['Ticker','2019_return']])
close2019

,Ticker,Close
3060,A,81.2400
3645,AA,29.6600
4755,AAC,2.8700
7816,AAL,34.6400
10877,AAME,2.8565
13938,AAN,54.8500
15304,AAOI,13.2400
18365,AAON,41.2600
21426,AAP,160.6200
24487,AAPL,174.9700


In [97]:
stock_return=st08.merge(st09,on='Ticker').merge(st10,on='Ticker').merge(st11,on='Ticker').merge(st12,on='Ticker').merge(st13,on='Ticker').merge(st14,on='Ticker').merge(st15,on='Ticker').merge(st16,on='Ticker').merge(st17,on='Ticker').merge(st18,on='Ticker').merge(st19,on='Ticker')

In [67]:
stock_return.to_csv('/Users/xinrucheng/Desktop/capstone/stock_return.csv', index = False)

In [98]:
stock_return

,Ticker,2008_return,2009_return,2010_return,2011_return,2012_return,2013_return,2014_return,2015_return,2016_return,2017_return,2018_return,2019_return
0,A,-0.573766,0.991667,0.319847,-0.159528,0.153460,0.367844,-0.000245,0.027805,0.121213,0.469954,0.009656,0.221654
1,AAL,-0.468729,-0.373868,1.068182,-0.498516,1.596154,0.840379,1.147188,-0.212417,0.140470,0.108926,-0.378750,0.104259
2,AAME,-0.521290,0.620253,0.537879,-0.033981,0.611012,0.330097,-0.004890,0.255639,-0.174349,-0.155556,-0.298064,0.175514
3,AAN,0.293454,0.039241,0.365430,0.304098,0.040837,0.026184,0.041658,-0.270237,0.451493,0.233410,0.059407,0.326802
4,AAON,0.107976,-0.022436,0.484606,0.111614,0.024584,1.261563,0.063500,0.039468,0.458808,0.104933,-0.042490,0.193520
5,AAP,-0.106041,0.202599,0.631204,0.050677,0.020382,0.509321,0.445451,-0.062791,0.134740,-0.414861,0.562934,0.028560
6,AAPL,-0.571685,1.453793,0.511315,0.243704,0.390506,0.162147,0.482183,-0.036808,0.150473,0.482642,-0.056417,0.134353
7,AAWW,-0.650194,0.960526,0.469984,-0.318859,0.124873,-0.096399,0.200097,-0.168271,0.287654,0.123563,-0.280034,0.308140
8,AAXN,-0.633588,-0.173585,0.053812,0.077895,0.706107,0.745228,0.671717,-0.352434,0.425882,0.086066,0.649698,0.266574
9,AB,-0.677853,0.408008,-0.128727,-0.382128,0.378845,0.299150,0.301408,-0.014149,0.060606,0.144421,0.205589,0.119807
